CS5242 Final Project : Prediction Notebook
===

*Murat Shagirov*

In [1]:
# %load_ext autoreload
# %autoreload 2

In [12]:
from os import path
import glob
import numpy as np
import pandas as pd
from skimage.io import imread

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from datautils import LoadTrainingData
from torch.utils.data import DataLoader
from torchvision import models, utils, transforms as T

from datautils import BatchUnnorm, Unnorm

from nn import predict_check, predict_test # prediction function

# check for CUDA device and set default dtype
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda:2')
dtype = torch.float32
print(f'device: {device}\ndtype: {dtype}')


# # # # # # # Set this before running # # # # # # # # # #
# Set CHECK_VAL_PRED to True if you want to verify train/validation set performances
CHECK_VAL_PRED = True # : True/False

# location of training ID-label pairs
train_csv_path = './datasets/train_label.csv' # : str

# Path to directory with training set images
train_img_path = './datasets/train_image/train_image/' # : str

# Path to save to/load models from
models_path = '../../dataDIR/cs5242/' # path : str
# model_fname = 'densenet121_ft_V1.pkl' # file name for desired model : str
model_fname = 'densenet121_ft_512px_v2.pkl' # file name for desired model : str

# !!! Path to directory with test dataset images !!!
test_path = './datasets/test_image/test_image/' # : str
# # # # # # # # # # # # # # # # # # # # # # # # # # # # #


device: cuda:2
dtype: torch.float32


In [4]:
# # Plotting tools and settings
# import matplotlib.pyplot as plt
# # for plotting figures (report)
# import matplotlib
# plt.style.use('ggplot')
# %matplotlib inline
# matplotlib.rcParams['figure.figsize'] = (15,5) # use larger for presentation
# matplotlib.rcParams['font.size']= 9 # use 14 for presentation

In [5]:
# Transforms
# unnorm = Unnorm() # unnormalize a single RGB image
# unnormb = BatchUnnorm() # unnormalize batch of images
img_size = 512 # Input image  size

# toPIL = T.ToPILImage()
# Training data transforms
transform = T.Compose([T.ToPILImage(),
                       T.RandomRotation((-3,3)),
                       T.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
                       T.RandomHorizontalFlip(),
                       T.ToTensor(),
                       T.ConvertImageDtype(dtype), 
                       T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Test and Val data transforms
val_transform = T.Compose([T.ToPILImage(),
                           T.Resize(img_size),
                           T.ToTensor(),
                           T.ConvertImageDtype(dtype),
                           T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
test_transform = val_transform
# Loss function
loss_func = nn.CrossEntropyLoss()


In [6]:
# # # # # # # # # # # # # # # # # # # #
# # Initiate and load desired model # #
# # # # # # # # # # # # # # # # # # # #

# Initiate default model (w/o weights) or download ImageNet pre-trained model from torchhub
model_ft = models.densenet121(pretrained=False,progress=False)
# change last FC layer
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Linear(num_ftrs, 3)

# Location for model's weights:
DENSENET121_path = path.join(models_path, model_fname)
model_ft.load_state_dict(torch.load(DENSENET121_path))
# set model mode for prediction:
model_ft.eval();

In [7]:
if CHECK_VAL_PRED:
    # Load and split original data into 80-20 Train-Val sets
    np.random.seed(42) #seed np RNG for consistency
    datasets = LoadTrainingData(train_csv_path, train_img_path, transform=transform,
                                split=True, train_percent=80, val_transform=val_transform)

    print(f"Training dataset: {len(datasets['train'])} samples.",
          f"\nValidation dataset: {len(datasets['val'])} samples.")

    # Batch sizes
    bsize_train = 4 # Training
    bsize_val = 4 # Val and Test

    # Prepare dataloaders
    # Set SHUFFLE==FALSE
    data_loaders = {'train' : DataLoader(datasets['train'], batch_size=bsize_train, shuffle=False, num_workers=0),
                    'val'   : DataLoader(datasets['val'],  batch_size=bsize_val, shuffle=False, num_workers=0)}  

    # Run prediction on training and validation datasets:
    losses, accuracies, pred_labels = predict_check(data_loaders, model_ft,device=device)
else:
    print('Skipping train/validation set performances.')

Training dataset: 931 samples. 
Validation dataset: 233 samples.
Loading: train
Loading: val
Losses: {'train': 0.006019095726112451, 'val': 0.10522011947352064}
Accuracies: {'train': tensor(0.9989), 'val': tensor(0.9785)}


In [13]:
pred_labels = predict_test(test_path, model_ft, test_transform, batch_size=4, device=device)

Found 292 images in test dataset folder: ./datasets/test_image/test_image
"0.png"
"1.png"
"2.png"
. . .
"289.png"
"290.png"
"291.png"]

Processing 73 test batches in total (batch_size=4).
Done.


In [18]:
# Write to csv file:
preds_df = pd.DataFrame(data=pred_labels).sort_values('ID') # to pd.DataFrame
preds_df.set_index('ID').to_csv('./test_submission.csv',sep=',')